## todo: 
* 接口改为fooocus
* 歌词:查找视频,选择视频,
* 独立程序(用:streamlit)

## 字幕(时序歌词)不再通过whisper识别, 而是直接通过剪映识别导出,然后根据原歌词调整

In [1]:
songPath = "其他歌/莲花轮回"

song = songPath+"/歌.mp3"
lyrics = songPath+"/歌词.txt" #原始歌词 
subtitles = songPath+"/字幕.srt" #时序歌词,字幕

width = 1024
height = 1024


## 直接读入: 字幕.srt

In [2]:
from datetime import datetime
def time_to_seconds(time_str):
    time_format = "%H:%M:%S,%f"
    time_obj = datetime.strptime(time_str, time_format)
    total_seconds = (time_obj.hour * 3600) + (time_obj.minute * 60) + time_obj.second + (time_obj.microsecond / 1000000)
    return total_seconds

def read_segments_from_srt(srt_file):
    segments = []
    with open(srt_file, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 4):
            segment = {}
            segment_id = lines[i].strip()
        
            if segment_id:
                segment['id'] = int(segment_id)
            else:
                print(i)
                return

            start_end = lines[i+1].strip().split(' --> ')
            segment['start'] = time_to_seconds(start_end[0])
            segment['end'] = time_to_seconds(start_end[1])
            segment['text1'] = lines[i+2].strip()
            segments.append(segment)

    for i in range(len(segments)-1):
        segments[i]['end'] = segments[i+1]['start']
       
    return segments

srt_file = subtitles
segments = read_segments_from_srt(srt_file)
print(segments)

[{'id': 1, 'start': 0.0, 'end': 6.166, 'text1': '轮回之莲'}, {'id': 2, 'start': 6.166, 'end': 11.066, 'text1': '词曲歌:平沢進'}, {'id': 3, 'start': 11.066, 'end': 16.966, 'text1': '图: xjq284'}, {'id': 4, 'start': 16.966, 'end': 21.4, 'text1': '金黄色的月亮'}, {'id': 5, 'start': 21.4, 'end': 25.2, 'text1': '相映在青草嫩叶的露珠上'}, {'id': 6, 'start': 25.2, 'end': 31.933, 'text1': '反射出万道光芒'}, {'id': 7, 'start': 31.933, 'end': 37.966, 'text1': '数亿个夜晚，你沉浸在你的梦中'}, {'id': 8, 'start': 37.966, 'end': 41.433, 'text1': '你困在网中，是梦的囚徒'}, {'id': 9, 'start': 41.433, 'end': 44.0, 'text1': '千万个月亮'}, {'id': 10, 'start': 44.0, 'end': 49.2, 'text1': '呼唤清晨，释放囚徒'}, {'id': 11, 'start': 49.2, 'end': 57.333, 'text1': '轮回之莲'}, {'id': 12, 'start': 57.333, 'end': 65.566, 'text1': '图: 自由行客'}, {'id': 13, 'start': 65.566, 'end': 69.1, 'text1': '无数个清晨组成的岁月'}, {'id': 14, 'start': 69.1, 'end': 75.033, 'text1': '来吧，自由行客，进入这无尽的时光'}, {'id': 15, 'start': 75.033, 'end': 81.0, 'text1': '回响吧，在无数的月夜中回响'}, {'id': 16, 'start': 81.0, 'end': 86.4, 'text1':

## 根据歌词text1生成图形提示词text2

### 用本地ChatGLM2

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os



llm_url = 'http://0.0.0.0:8000' #本地LLM API地址   =>llmwebui

def prompt_generation(text1):
    # 发送HTTP POST请求
    headers = {
	'accept': 'application/json',
	'Content-Type': 'application/json'
    }
    
    data = {'prompt':'把下面的内容翻译为英文并扩展一下：'+text1}
    json_data = json.dumps(data)
 
    # 发送HTTP POST请求
    response = requests.post(llm_url, data=json_data, headers=headers)

    result_json = json.loads(response.text)
 
    # 输出结果
    if 'response' in result_json:
        return result_json['response']
    else:
        print("Unexpected result:", result_json)
        return None

for i in range(len(segments)):
    segments[i]['text2'] = prompt_generation(segments[i]['text1'])

## 用openai

In [ ]:
import requests
import openai
import io
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

def prompt_generation(text1):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "根据下面的内容描述，生成一幅梵高画风的空灵画面, 用英文表示:"},
            {"role": "user", "content": text1}
        ]
    )
    return response["choices"][0]["message"]["content"]


for i in range(0,len(segments)):
    segments[i]['text2'] = prompt_generation(segments[i]['text1'])

In [ ]:
segments

## 根据text2生成图片到images目录下
novel_dict对sdxl要从新调整

## SDXL

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os

sd_url = "http://127.0.0.1:7860/sdapi/v1/txt2img"  #  SDXL

prompt = "best quality,masterpiece, Extremely Fine and Beautiful,fantasy style, a Van Gogh-inspired painting:  "
payload = { #SDXL: 
    "prompt": [],
    "width": width, #最大:1048576, 缺省:512
    "height": height,
    "steps": 30,
    "cfg_scale": 7,
    "sampler": "UiPC",#缺省: k_dpmpp_2m
    'batch_size': 1,    
    "send_images": "true",
    "save_images": "true",
}


def gen_image(i):    
    payload['prompt']=segments[i]['text2']+prompt
    
    print(payload['prompt'])
    html = requests.post(sd_url, data=json.dumps(payload))
    print(html)
    image_response = json.loads(html.text)
    image_bytes = base64.b64decode(image_response['images'][0])
    image = Image.open(io.BytesIO(image_bytes))
    image_file = songPath+'/images/'+str(segments[i]['id'])+'.jpg'
    image.save(image_file)
    segments[i]['image_file'] = image_file

for i in range(0,len(segments)):
    gen_image(i)


#gen_image(28)

## Fooocus

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os

sd_url = "http://127.0.0.1:8888/v1/generation/text-to-image"  # fooocus

prompt = "best quality,masterpiece, Extremely Fine and Beautiful,fantasy style, a Van Gogh-inspired painting:  "

payload = { #fooocus
  "prompt": "",
  "negative_prompt": "",
  "style_selections": [
    "Fooocus V2",
    "Fooocus Enhance",
    "Fooocus Sharp"
  ],
  "performance_selection": "Speed",
  "aspect_ratios_selection": "1024×1024",
  "image_number": 1,
  "image_seed": -1,
  "sharpness": 2,
  "guidance_scale": 7,
  "base_model_name": "sd_xl_base_1.0_0.9vae.safetensors",
  "refiner_model_name": "sd_xl_refiner_1.0_0.9vae.safetensors",
  "refiner_switch": 0.8,
  "loras": [
    {
      "model_name": "sd_xl_offset_example-lora_1.0.safetensors",
      "weight": 0.5
    }
  ],
  "advanced_params": {
    "adm_scaler_positive": 1.5,
    "adm_scaler_negative": 0.8,
    "adm_scaler_end": 0.3,
    "refiner_swap_method": "joint",
    "adaptive_cfg": 7,
    "sampler_name": "dpmpp_2m_sde_gpu",
    "scheduler_name": "karras",
    "overwrite_step": -1,
    "overwrite_switch": -1,
    "overwrite_width": -1,
    "overwrite_height": -1,
    "overwrite_vary_strength": -1,
    "overwrite_upscale_strength": -1,
    "mixing_image_prompt_and_vary_upscale": "false",
    "mixing_image_prompt_and_inpaint": "false",
    "debugging_cn_preprocessor": "false",
    "controlnet_softness": 0.25,
    "canny_low_threshold": 64,
    "canny_high_threshold": 128,
    "inpaint_engine": "v1",
    "freeu_enabled": "false",
    "freeu_b1": 1.01,
    "freeu_b2": 1.02,
    "freeu_s1": 0.99,
    "freeu_s2": 0.95
  },
  "require_base64": "false",
  "async_process": "false"
}



def gen_image(i):    
    payload['prompt']=segments[i]['text2']+prompt
    
    print(payload['prompt'])
    html = requests.post(sd_url, data=json.dumps(payload))
    image_response = json.loads(html.text)
    response = requests.get(image_response[0]['url'])
    image_bytes = response.content
    image = Image.open(io.BytesIO(image_bytes))
    image_file = songPath+'/images/'+str(segments[i]['id'])+'.jpg'
    image.save(image_file)
    segments[i]['image_file'] = image_file

for i in range(0,len(segments)):
    gen_image(i)


#gen_image(28)

In [ ]:
image_response

## 直接更新图像

In [3]:
for i in range(len(segments)):
    image_file = songPath+'/images/'+str(segments[i]['id'])+'.jpg'
    segments[i]['image_file'] = image_file



In [ ]:

segments


# 根据字幕和images生成影片

In [4]:
from moviepy.editor import *
import cv2



def fl_zoom(gf, t, start_scale=1.0, end_scale=1.2):
    image = gf(t)
    if image.size == 0:
        print(f"Empty image at time {t}")
        return image
    scale = start_scale + (end_scale - start_scale) * t / duration
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    return cv2.resize(image, new_size)

def check_image_file(image_file):
    if os.path.exists(image_file):
        return image_file

    jpeg_file = image_file.lower().replace('.jpg', '.jpeg')
    if os.path.exists(jpeg_file):
        return jpeg_file

    png_file = image_file.lower().replace('.jpg', '.png')
    if os.path.exists(png_file):
        return png_file

    return None


font = "/System/Library/Fonts/PingFang.ttc" 
clips = []
duration=1

for i in range(len(segments)):
#for i in range(0,30):
    image_file = segments[i]['image_file']
    image_file = check_image_file(image_file)
    start = segments[i]['start']
    end = segments[i]['end']
    duration = end - start
    text1 = segments[i]['text1']
    print(i, image_file, duration, text1)

    image_clip = ImageClip(image_file)
    image_clip = image_clip.set_duration(duration)  # 设置时长
    image_clip = image_clip.fl(fl_zoom)
  

    text_clip = TextClip(text1,fontsize=54,font=font,color='yellow', bg_color='black')
    #text_clip = text_clip.set_position(('center','bottom'))
    text_clip = text_clip.set_position(('center',0.8), relative=True)
    text_clip = text_clip.set_duration(duration)
    text_clip = text_clip.set_opacity(0.6)
    
    composite_clip = CompositeVideoClip([image_clip, text_clip])
    clips.append(composite_clip)
final_clip = concatenate_videoclips(clips)
audio_clip = AudioFileClip(song)
final_clip = final_clip.set_audio(audio_clip)
final_clip.write_videofile(songPath+"/歌.mp4", fps=24, audio_codec="aac")




0 其他歌/莲花轮回/images/1.jpeg 6.166 轮回之莲
1 其他歌/莲花轮回/images/2.jpeg 4.9 词曲歌:平沢進
2 其他歌/莲花轮回/images/3.jpg 5.9 图: xjq284
3 其他歌/莲花轮回/images/4.jpeg 4.4339999999999975 金黄色的月亮
4 其他歌/莲花轮回/images/5.jpeg 3.8000000000000007 相映在青草嫩叶的露珠上
5 其他歌/莲花轮回/images/6.jpeg 6.7330000000000005 反射出万道光芒
6 其他歌/莲花轮回/images/7.jpeg 6.033000000000001 数亿个夜晚，你沉浸在你的梦中
7 其他歌/莲花轮回/images/8.jpeg 3.4669999999999987 你困在网中，是梦的囚徒
8 其他歌/莲花轮回/images/9.jpeg 2.567 千万个月亮
9 其他歌/莲花轮回/images/10.jpeg 5.200000000000003 呼唤清晨，释放囚徒
10 其他歌/莲花轮回/images/11.jpeg 8.132999999999996 轮回之莲
11 其他歌/莲花轮回/images/12.jpg 8.233000000000004 图: 自由行客
12 其他歌/莲花轮回/images/13.jpeg 3.533999999999992 无数个清晨组成的岁月
13 其他歌/莲花轮回/images/14.jpeg 5.933000000000007 来吧，自由行客，进入这无尽的时光
14 其他歌/莲花轮回/images/15.jpeg 5.966999999999999 回响吧，在无数的月夜中回响
15 其他歌/莲花轮回/images/16.jpeg 5.400000000000006 轮回
16 其他歌/莲花轮回/images/17.jpeg 5.432999999999993 莲花
17 其他歌/莲花轮回/images/18.jpeg 5.367000000000004 轮回之莲
18 其他歌/莲花轮回/images/19.jpeg 4.332999999999998 绽放吧, 轮回之莲
19 其他歌/莲花轮回/images/20.jpeg 4.0 绽放吧, 轮回之莲
20 其

MoviePy - Done.
Moviepy - Writing video 其他歌/莲花轮回/歌.mp4



Moviepy - Done !
Moviepy - video ready 其他歌/莲花轮回/歌.mp4


In [ ]:
segments[39]['text1']='沙丘 词曲歌:刀郎 图:xjq284'

## 由于mac字体有问题, 比如远等字无法显示, 去掉字幕

In [ ]:

from moviepy.editor import *
import cv2



def fl_zoom(gf, t, start_scale=1.0, end_scale=1.1):
    image = gf(t)
    if image.size == 0:
        print(f"Empty image at time {t}")
        return image
    scale = start_scale + (end_scale - start_scale) * t / duration
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    return cv2.resize(image, new_size)


font = "/System/Library/Fonts/PingFang.ttc" 
clips = []
duration=1

for i in range(len(segments)):
    image_file = segments[i]['image_file']
    start = segments[i]['start']
    end = segments[i]['end']
    duration = end - start
    text1 = segments[i]['text1']
    print(i, image_file, duration, text1)

    image_clip = ImageClip(image_file)
    image_clip = image_clip.set_duration(duration)  # 设置时长
    image_clip = image_clip.fl(fl_zoom)
  
    composite_clip = CompositeVideoClip([image_clip])
    clips.append(composite_clip)
final_clip = concatenate_videoclips(clips)
#audio_clip = AudioFileClip(song)
#final_clip = final_clip.set_audio(audio_clip)
final_clip.write_videofile(songPath+"/歌.mp4", fps=24, audio_codec="aac")






## 根据字幕片段,生成视频查询关键词
segments

In [ ]:
import requests
import json
import base64
from PIL import Image
import io
import os

llm_url = 'http://0.0.0.0:8000' #本地LLM API地址

sys='''system_prompt:
  # Instructions

  You're a video research expert. The user will give you the timed captions of a video they will make, and you will give back a list of couples of text search queries that will be used to search for background video footage, and the time t1 and t2 when it will be shown in the video.
  # Output format
  The format will be JSON parasable and look like this:
  [[[0.0, 4.4], ["Dog barking", "Dog angry", Canine angry"]], [[4.4, 7.8], "bone", "pet food", "food", "canine"], ect...

  # Time periods t1 and t2
  Time periods t1 and t2 must always be consecutive, and last between 4 to 5 seconds, and must cover the whole video.
  For example, 	[0, 2.5] <= IS BAD, because 2.5-0 = 2.5 < 3
              [0, 11]  <= IS BAD, because 11sec > 5 sec
              [0, 4.2] <= IS GOOD
              
  # Query search string list
  YOU ALWAYS USE ENGLISH IN YOUR TEXT QUERIES
  As you have seen above, for each time period you will be tasked to generate 3 strings that will be searched on the video search engine, to find the appropriate clip to find.
  Each string has to be between ONE to TWO words.
  Each search string must DEPICT something visual.
  The depictions have to be extremely visually concrete, like `coffee beans`, or `dog running`.
  'confused feelings' 			<= BAD, because it doesn't depict something visually
  'heartbroken man' 			<= GOOD, because it depicts something visual. 
  'man feeling very lonely' 	<= BAD, because it contains 4 words.
  The list must always contain 3 query searches.
  ['Sad person']					<= BAD, because it's one string
  ['Sad person', 'depressed man', 'depressed person'] <= GOOD, because it's 3 strings
  ['Une Pomme', 'un enfant qui rit', 'une personne heureuse'] <= BAD, because the text query is NOT in english
'''

def words_generation(text1):
    # 发送HTTP POST请求
    
    #data = {'system_prompt':sys,  'chat_prompt': "Timed captions:"+text1+"Video search queries"}
    data = {'prompt':sys+'chat_prompt: Timed captions:'+text1+'Video search queries'}
    
    json_data = json.dumps(data)

    # 发送HTTP POST请求
    response = requests.post(llm_url, data=json_data, headers={'Content-Type': 'application/json'})
    result_json = json.loads(response.text)
 
    # 输出结果
    if 'response' in result_json:
        return result_json['response']
    else:
        print("Unexpected result:", result_json)
        return None

for i in range(len(segments)):
    segments[i]['text3'] = words_generation(segments[i]['text1'])

## 根据歌词片段, 生成视频clip

In [ ]:
import requests
from moviepy.video.io.VideoFileClip import VideoFileClip

def search_videos(query_string, landscape=True):
    url = "https://api.pexels.com/videos/search"
    headers = {
        "Authorization":"RQdKQPXlzy3URM55teyKcU5taQBPE5J9pFlh7WNG4tcQASjAz0Nkn5l6"
    }
    params = {
        "query": query_string,
        "orientation": "landscape" if landscape else "portrait",
        "per_page": 15
    }

    response = requests.get(url, headers=headers, params=params)
    json_data = response.json()

    return json_data


def getBestVideo(query_string, orientation_landscape=True, used_vids=[]):
    vids = search_videos(query_string, orientation_landscape)
    videos = vids['videos']  # Extract the videos list from JSON

    # Filter and extract videos with width and height as 1920x1080 for landscape or 1080x1920 for portrait
    if orientation_landscape:
        filtered_videos = [video for video in videos if video['width'] >= 1920 and video['height'] >= 1080 and video['width']/video['height'] == 16/9]
    else:
        filtered_videos = [video for video in videos if video['width'] >= 1080 and video['height'] >= 1920 and video['height']/video['width'] == 16/9]

    # Sort the filtered videos by duration in ascending order
    sorted_videos = sorted(filtered_videos, key=lambda x: abs(15-int(x['duration'])))

    # Extract the top 3 videos' URLs
    for video in sorted_videos:
        for video_file in video['video_files']:
            if orientation_landscape:
                if video_file['width'] == 1920 and video_file['height'] == 1080:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
            else:
                if video_file['width'] == 1080 and video_file['height'] == 1920:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
    print("NO LINKS found for this round of search with query :", query_string)
    return None

In [ ]:
vurl=getBestVideo('sad eyes',1).split('&')[0]
clip = VideoFileClip(vurl)
clip.write_videofile('temp.mp4')

# 转换heic文件为jpg文件

In [ ]:


def convert_heic_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".heic"):
            heic_file = os.path.join(directory, filename)
            jpg_file = os.path.splitext(heic_file)[0] + ".jpg"
            ! magick convert {heic_file} {jpg_file}

# 指定目录路径
directory_path = "/Users/xjq284/Pictures/xjq"

# 调用函数进行转换
convert_heic_to_jpg(directory_path)

In [ ]:
import os
from PIL import Image

def convert_heic_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".heic"):
            heic_file = os.path.join(directory, filename)
            jpg_file = os.path.splitext(heic_file)[0] + ".jpg"
            
        
            
            # 将HEIC图像转换为PIL图像对象
            image = Image.open(heic_file)
            
            # 保存为JPG格式
            image.save(jpg_file, "JPEG")
            
            # 删除原始的HEIC文件
            os.remove(heic_file)

# 指定目录路径
directory_path = "/Users/xjq284/Pictures/xjq"

# 调用函数进行转换
convert_heic_to_jpg(directory_path)

# 录音5分钟

In [ ]:
import os
import sounddevice as sd
from scipy.io import wavfile
from pydub import AudioSegment

def convert_wav_to_mp3(wav_file, mp3_file):
    # 读取.wav文件
    audio = AudioSegment.from_wav(wav_file)

    # 将音频导出为MP3格式
    audio.export(mp3_file, format="mp3")

def record_audio(duration, filename):
    # 设置录音参数
    sample_rate = 44100  # 采样率
    channels = 1  # 声道数

    # 开始录音
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels)

    # 等待录音完成
    sd.wait()

    # 保存录音为.wav文件
    wavfile.write(filename, sample_rate, recording)

# 设置录音时长和保存文件名
duration = 5*60  # 录音时长（秒）
filename = "recording"  # 保存文件名

wav_file = "/Users/xjq284/myvoice/"+filename+".wav"
mp3_file = "/Users/xjq284/myvoice/"+filename+".mp3"

# 调用录音函数
record_audio(duration, wav_file)
convert_wav_to_mp3(wav_file, mp3_file)
os.remove(wav_file)


In [ ]:
from pydub import AudioSegment

def convert_wav_to_mp3(wav_file, mp3_file):
    # 读取.wav文件
    audio = AudioSegment.from_wav(wav_file)

    # 将音频导出为MP3格式
    audio.export(mp3_file, format="mp3")

# 指定输入的.wav文件和输出的MP3文件
wav_file = "recording2.wav"
mp3_file = "recording2.mp3"

# 调用函数进行转换


从一个"
0:06
关于权重和神经网络
0:08
权重是神经网络的参数
0:10
网络的参数"格式文件读入, 然后以以下格式"
1
0:06-->0:08
关于权重和神经网络
2
0:08-->0:10
权重是神经网络的参数
3
0:10-->0:12
网络的参数"
写入另外一个文件

# 把youtube类型字幕转换为srt字母

In [ ]:
from datetime import datetime

def format_time(time_str):
    time_format = "%M:%S"
    time_obj = datetime.strptime(time_str, time_format)
    return time_obj.strftime("%H:%M:%S,%f")[:-3]

def convert_to_srt(input_file, output_file):
    with open(input_file, 'r') as file:
        content = file.read().strip().split('\n')

    srt = ''

    for i in range(0, len(content)-2, 2):
        start_time = format_time(content[i])
        end_time = format_time(content[i+2])
        if len(content[i+1]) > 1:
            text = content[i+1]
        else:
            text = ""
        srt += f"{i//2+1}\n{start_time} --> {end_time}\n{text}\n\n"

    with open(output_file, 'w') as file:
        file.write(srt)

# 指定输入文件和输出文件
input_file = "1.srt"
output_file = "2.srt"

# 调用函数进行转换
convert_to_srt(input_file, output_file)


In [ ]:
import requests
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

# ChatGPT的API端点
api_endpoint = 'https://api.openai.com/v1/engines/davinci/completions'  # 使用davinci模型
api_endpoint = 'https://api.openai.com/v1/engines/text-davinci-002/completions'
api_endpoint = 'https://api.openai.com/v1/engines/gpt-3.5-turbo-instruct/completions'



# 请求的输入消息
user_message = '请用英文,把下面的文字转换为精彩的易于画画的描述,: 往日错过的渡口'

# 发送POST请求
response = requests.post(
    api_endpoint,
    headers={
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    },
    json={
        'prompt': user_message,
        'max_tokens': 250  # 可选参数，限制生成的文本长度
    }
)

# 解析并打印ChatGPT的回复
data = response.json()
return(data['choices'][0]['text'])




In [ ]:
import requests
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "根据下面的内容描述，生成一副生动的简洁中国画, 用英文表示:"},
        {"role": "user", "content": "靠着些微弱的光亮"}
    ]
)



In [ ]:
response["choices"][0]["message"]["content"]

In [ ]:
import os

def rename_jpeg_to_jpg(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.png'):
                old_path = os.path.join(root, file)
                new_path = os.path.join(root, file.lower().replace('.png', '.jpg'))
                os.rename(old_path, new_path)

# 指定当前目录
current_directory = '/Users/xjq284/Fooocus/outputs'

# 调用函数进行文件重命名
rename_jpeg_to_jpg(current_directory)